# USG grants crawl
## Agency-specific 

### Previously

In the previous chapter we looked at how often certain open science infrastructure-related terms were featured in grants from grants listed on [grants.gov](https://www.grants.gov/web/grants) and how often they co-occured.

In this chapter we'll move on to looking how federal agencies play into this as well.  Specifically, we'll look at which terms are occuring in which agencies grants.

### Loading the database once more

Let's begin by loading up the database provided by the website, which is stored in an xml format.

In [3]:
# import our helper functions
import sys
import os
sys.path.insert(0, os.path.abspath('../../src'))
sys.path
import grantsGov_utilities as grantsGov_utilities

# local data storage directory
localDataDir='inputData'

grantsDF=grantsGov_utilities.detectLocalGrantData(localPath='../../'+localDataDir,forceDownload=True)
grantsDF

No local grant data XML file found
41283 bytes file downloaded from
https://www.grants.gov/extract/GrantsDBExtract20240709v2.zip
Saved to ../../inputData


BadZipFile: File is not a zip file

## Cleaning
As before, we need to do a bit of cleaning, so lets do a more comprehensive version of that here.

Note:  This may take a moment

In [ ]:
grantsDF=grantsGov_utilities.prepareGrantsDF(grantsDF, repair=True)
grantsDF

NameError: name 'grantsDF' is not defined

### Keywords and terms

Although we aren't going to inspect the keywords and agencies on their own this time, we still need to collect them.  Once we have loaded them, we can determine which words are occuring in which grants, and which agencies those grants are associated with.  The resulting information can be placed in a dictionary, where the relevant information can be accessed by using the [`tuple`](https://www.w3schools.com/python/python_tuples.asp) corresponding to the desired agency and keyword (e.g. (`[agency]`,`[keyword]`)).

In [ ]:
import seaborn as sns
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

# open the keywords file
with open('../keywords.txt', 'r') as f:
    keywords = f.read()

# split it into a list.  Each term is kept on a separate line
keywords=keywords.split('\n')

# find the grants that are associated with thse keywords
grantFindsOut=grantsGov_utilities.searchGrantsDF_for_keywords(grantsDF,keywords)
# find the agencies associated with these
grantAgenciesOut=grantsGov_utilities.grants_by_Agencies(grantsDF)

# get a dataframe with the keyword by agency information
keywordsByAgency_DF=grantsGov_utilities.evalGrantCoOccurrence([grantFindsOut,grantAgenciesOut],formatOut='dataframe')
# get the counts for all of these
keywordsByAgency_count_DF=keywordsByAgency_DF.applymap(lambda x: len(x))

### A small wait

Because the previous analysis isn't coded particularly efficient, it can take a moment to complete.  Part of this has to do with the inefficiency required to index back in to the database, as well as the inefficient storage method for the information we are getting (i.e. appending to lists in a large dictionary)

In any case, once we have the relevant data structure we can look at which agencies are using which terms, and also receive an ouput of the [grants.gov](https://www.grants.gov/web/grants) IDs associated with those grants

In [ ]:
# chat-davinci-002 prompt
# an iteractive jupyer notebook widget that returns two subplot windows.  The input is a numerical matrix.  The interface features two dropdown menus that allow you to select a row (i) and column (j) from the matrix.  On the left side of the subplot outputs, a matrix heatmap plotting the numerical data.  On the right side of the subplot outputs, a blank plot that is used to display text indicating the value found in the specific matrix (i,j) entry selected in the dropdown menus.

import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import seaborn as sns
from IPython.display import clear_output
from matplotlib.colors import LogNorm

def heatmap_plot(matrix, heatmap_ax, row, column):
    """
    Plots the heatmap with a crosshair at the desired location
    """
    # if row is empty, default to column
    if row == '':
        row = column
    # if column is empty, default to row
    if column == '':
        column = row
    # if both are empty, no outline
    if row == '' and column == '':
        row = 0
        column = 0
    # if both are not empty, only highlight the relevant cell
    if row != '' and column != '':
        row = row
        column = column
    # create the heatmap plot
    sns.heatmap(matrix, ax=heatmap_ax, norm=LogNorm(),cmap='viridis', cbar=True, xticklabels=list(keywordsByAgency_count_DF.columns), yticklabels=keywords, cbar_kws={'label': 'Grant count\n(log-scale)'})
    # create the outline
    heatmap_ax.axvline(x=column+.5, color='red', linewidth=2)
    heatmap_ax.axhline(y=row+.5, color='red', linewidth=2)
    heatmap_ax.set(xlabel='Agency', ylabel='OS Keyword')
    # show the plot

def plot_list(axis, list_of_text, font_size=None, font_color='black', font_family='sans-serif') :
    """
    A function for plotting a list of text elements evenly across a passed in axis.  The function begins by taking in the passed in axis and measuring the space available.  The function then uses those dimensions to determine both the font size and how the list elements should be split into rows and columns so as to take up the maximum amount of space available within the axis, without overlapping.  The function then plots those list elements to the axis space.  Finally the plot is displayed.  The function does not alter the size of the input axes or resultant figure.

    Parameters
    ----------
    axis : matplotlib.axes.Axes
        The axis to plot the list of text elements to.
    list_of_text : list
        A list of text elements to plot to the axis.
    font_size : int, optional
        The font size to use for the text elements.  If not passed in, the function will calculate the font size based on the size of the axis.
    font_color : str, optional
        The color of the text elements.  The default is 'black'.
    font_family : str, optional
        The font family to use for the text elements.  The default is 'sans-serif'.

    testBox:
    
    aaaaa
    aaaaa
    aaaaa
    

    Returns
    -------
    None.

    """
    import math


    # get the axis dimensions
    #first get the figure handle
    fig=axis.get_figure()
    bbox = axis.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    axis_width, axis_height = bbox.width, bbox.height
    # returns in pixels, for some reason
    #axis_width = axis.get_window_extent().width
    #axis_height = axis.get_window_extent().height

    # no need to get units for these axes sizes as we can safely assume they are in inches.
    spaceNum=4

    #assumed aspect ratio, how many characters can you fit along x amount of space vertically : horizontally; see text box for demo
    textAspectRatio=5/5
    
    # calculate the maximum number of characters in the text elements and use this to establish the expected character width of columns
    max_characters = max([len(x) for x in list_of_text])

    # create a list of spaces to add to the end of each list element
    spaces = [' ' * (max_characters - len(x)) for x in list_of_text]

    # join the list of text elements with the list of spaces
    list_of_text = [x + y for x, y in zip(list_of_text, spaces)]

    nonSpaceCount=len(list_of_text) - list_of_text.count(' ')
    spacaceCount=list_of_text.count(' ')
    #estimate total character footprint
    charFootprint=nonSpaceCount+(spacaceCount/2)
    #quasi math: rows=3/5*cols; squareform=cols^2; totalChars=(3/5*cols)*cols
    colNum=math.ceil(math.sqrt(charFootprint*5/3))
    rowNum=math.ceil(colNum*(textAspectRatio))
    
    #element_per_row=math.ceil(colNum/(max_characters+2))
    #get the nearest root that's equal to or greter than len(list_of_text) root
    math.ceil(charFootprint/rowNum)

    mergedText=''
    # conditional appending
    for iTextIndex, iTextElements in enumerate(list_of_text):
        #if it's divisible by the number of elements per row
        if (iTextIndex+1) % math.ceil(charFootprint/rowNum) == 0:
            mergedText=mergedText + iTextElements + '\n'
        else:
            mergedText=mergedText + iTextElements + spaceNum * ' '

    #how many chars per row, spaces only count as half, it seems
    #rowCharNumber= (rows_element_num * (max_characters  + math.ceil(spaceNum/2)))-math.ceil(spaceNum/2)  

    #72 seems to not actuall be = to one inch
    fontScaleFactor=.3

    # calculate the maximum allowable font size based on the both the height and width axes, such that no text from list_of_text will exceed the axes boundaries.  Assume 1 point of font is equal to 1/72 inches.
    maxWidthFont=(axis_width / (colNum / (72 * fontScaleFactor)))
    maxHeightFont=(axis_height /( rowNum / (32* fontScaleFactor)))
    max_font_size = min([maxWidthFont, maxHeightFont])

    # if a font size was passed in, use it.  Otherwise, use the calculated font size.
    if font_size is None :
        font_size = max_font_size

    # plot the list of text elements to the axis
    axis.text(0, 0, mergedText, fontsize=font_size, color=font_color, family=font_family)

    # display the plot
    plt.show()


# create a function that updates the heatmap
def heatmap_and_text(countMatrix,rowSelect,columnSelect):
    """
    Plots both the heatmap and the textbox of grants in a 1 by 2 subplot
    """


    fig, ax = plt.subplots(2, 1, figsize=(10, 20))

    # plot the heatmap
    heatmap_plot(countMatrix, heatmap_ax=plt.gcf().get_axes()[0], row=rowSelect, column=columnSelect)
    keyTuple=tuple([col_menu.value,row_menu.value])
    try:
        list_to_plot=dataHolder[keyTuple]
    except:
        list_to_plot=['No grants found']
    plot_list(plt.gcf().get_axes()[1],list_to_plot)
    # show the plot

def update_plots(rowSelectName,columnSelectName):
    """
    Performs the updating
    """
    
    rowIndex=list(keywords).index(rowSelectName)
    colIndex=list(keywordsByAgency_count_DF.columns).index(columnSelectName)
    heatmap_and_text(keywordsByAgency_count_DF.values,rowIndex,colIndex)
    
    
# link the dropdown menus to the update functions
#row_menu.observe(update_heatmap, names='value')
#col_menu.observe(update_heatmap, names='value')
# display the widgets
#display(row_menu)
#display(col_menu)

# update the heatmap
#update_heatmap(None)
# create a dropdown menu for the rows
row_menu = widgets.Dropdown(
    options=list(keywords),
    #value=,
    description='Row:',
    disabled=False,
)
# create a dropdown menu for the columns
col_menu = widgets.Dropdown(
    options=list(keywordsByAgency_count_DF.columns),
    #value='',
    description='Column:',
    disabled=False,
    )


%matplotlib inline
from ipywidgets import interact
#establishes interactivity
interact(update_plots,rowSelectName=row_menu,columnSelectName=col_menu)

interactive(children=(Dropdown(description='Row:', options=('metadata', 'interoperable', 'reusable', 'data reu…

<function __main__.update_plots(rowSelectName, columnSelectName)>

### Interacting with the plot 

The widget should allow you to select which terms to work with.  For the moment (i.e. early stages of this notebook) the interface is relatively rudamentary but the heatmap plot should feature a crosshair indicating which agency and term you are looking at.  The plot beneath that should inclde a list of the grant.gov IDs.  In many cases no grants are found meeting the criteria, and so a large text indicator should appear stating this.  However in the event that grants are found, they should be listed.  Currently the text scaling for this feature is rudamentary, and so if too many are found their font might be extremely small (future [modifications](https://stackoverflow.com/questions/55729075/matplotlib-how-to-autoscale-font-size-so-that-text-fits-some-bounding-box) could adress this).  Additionally, the text elements themselves may be [capable of being hyperlinks](https://matplotlib.org/stable/gallery/misc/hyperlinks_sgskip.html).

In any case, we can also attempt to replicate this process and look at the value of the grants as well.  As before, this computation will take a moment.

In [ ]:
#now do it again with value

# see if we can do it cleverly with applymap
keywordsByAgency_sumValue_DF=keywordsByAgency_DF.applymap(lambda x: 0+np.sum([grantsDF['EstimatedTotalProgramFunding'].loc[grantsDF['OpportunityID'].eq(igrantID)].values for igrantID in x]))

### Plotting the values

We'll reuse much of the same code as we did before, except this time we'll be redefining the section where we took in the count matrix.  The interactivity of the resulting plot should be quite the same as the previous one.

Keep in mind though, that there may be errors in how the grant values have been reported, and so the reultant trends should be taken with a grain of salt.


In [ ]:
# redefining for new matrix input
def update_plots(rowSelectName,columnSelectName):
    """
    Performs the updating
    """
    
    rowIndex=list(keywords).index(rowSelectName)
    colIndex=list(list(keywordsByAgency_count_DF.columns)).index(columnSelectName)
    heatmap_and_text(keywordsByAgency_sumValue_DF.values,rowIndex,colIndex)
    
# update the heatmap
#update_heatmap(None)
# create a dropdown menu for the rows
row_menu = widgets.Dropdown(
    options=list(grantFindsOut.keys()),
    #value=,
    description='Row:',
    disabled=False,
)
# create a dropdown menu for the columns
col_menu = widgets.Dropdown(
    options=list(keywordsByAgency_count_DF.columns),
    #value='',
    description='Column:',
    disabled=False,
    )

# redefining for new axes labels
def heatmap_plot(matrix, heatmap_ax, row, column):
    """
    Plots the heatmap with a crosshair at the desired location
    """
    # if row is empty, default to column
    if row == '':
        row = column
    # if column is empty, default to row
    if column == '':
        column = row
    # if both are empty, no outline
    if row == '' and column == '':
        row = 0
        column = 0
    # if both are not empty, only highlight the relevant cell
    if row != '' and column != '':
        row = row
        column = column
    # create the heatmap plot
    sns.heatmap(matrix, ax=heatmap_ax, norm=LogNorm(),cmap='viridis', cbar=True, xticklabels=list(keywordsByAgency_count_DF.columns), yticklabels=keywords, cbar_kws={'label': 'Grant value\n(USD, log-scale)'})
    # create the outline
    heatmap_ax.axvline(x=column+.5, color='red', linewidth=2)
    heatmap_ax.axhline(y=row+.5, color='red', linewidth=2)
    heatmap_ax.set(xlabel='Agency', ylabel='OS Keyword')
    # show the plot

%matplotlib inline
from ipywidgets import interact
#establishes interactivity
interact(update_plots,rowSelectName=row_menu,columnSelectName=col_menu)

interactive(children=(Dropdown(description='Row:', options=('metadata', 'interoperable', 'reusable', 'data reu…

<function __main__.update_plots(rowSelectName, columnSelectName)>